In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
tqdm.pandas() 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
random_state = 2023

In [ ]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../data/xy_train.csv'
test_path  = '../data/xy_test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

In [ ]:
print(dt.shape, dt_test.shape)

In [5]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt['is_test'] = 0
dt_test['is_test'] = 1
concat = pd.concat([dt, dt_test])     # 하나의 데이터로 만들어줍니다.

# 좌표

In [ ]:
concat[['좌표X', '좌표Y']].isnull().sum()

In [7]:
api_key = ''

In [8]:
def get_coordinates_with_fallback(region, bonbun, bubun, api_key):
    """
    주소를 단계적으로 축소해가며 좌표를 검색합니다.
    
    Parameters:
    - region (str): 행정구역 (시/도 + 구/군 + 읍/면/동)
    - bonbun (int): 본번
    - bubun (int): 부번
    - api_key (str): Kakao Maps REST API 키
    
    Returns:
    - tuple: (latitude, longitude) 또는 None (오류 시)
    """
    # 주소를 축소하는 순서대로 리스트 생성
    addresses = []
    # if bubun > 0:
    addresses.append(f"{region} {bonbun}-{bubun}")  # 본번과 부번 포함
    addresses.append(f"{region} {bonbun}")  # 본번만 포함
    addresses.append(region)  # 행정구역만 포함

    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}

    for address in addresses:
        params = {"query": address}
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            result = response.json()

            if result["documents"]:
                # 첫 번째 결과의 좌표 반환
                x = result["documents"][0]["x"]  # 경도
                y = result["documents"][0]["y"]  # 위도
                # print(f"주소 검색 성공: {address}")
                return float(y), float(x)
            else:
                # print(f"주소를 찾을 수 없습니다: {address}")
                pass

        except requests.exceptions.HTTPError as e:
            print(f"HTTP 오류 발생: {e}")
        except Exception as e:
            print(f"기타 오류 발생: {e}")
    
    # 모든 시도가 실패하면 None 반환
    return None

In [ ]:
unique_addresses = concat[['시군구', '본번', '부번']].drop_duplicates()
unique_addresses

In [ ]:
unique_addresses['coordinates'] = unique_addresses.progress_apply(
    lambda row: get_coordinates_with_fallback(row['시군구'], row['본번'], row['부번'], api_key=api_key), axis=1
)

In [ ]:
concat = concat.merge(unique_addresses, on=['시군구', '본번', '부번'], how='left')
concat.head()

In [ ]:
concat['좌표Y'] = concat['coordinates'].astype('str').str.split(',', expand=True)[0].str.split('(', expand=True)[1].astype('float')
concat['좌표X'] = concat['coordinates'].astype('str').str.split(',', expand=True)[1].str.split(')', expand=True)[0].astype('float')
concat.head()

In [12]:
concat.drop(columns=['coordinates'], axis=1, inplace=True)

In [ ]:
concat[['좌표X', '좌표Y']].isnull().sum()

In [ ]:
plt.scatter(concat['좌표X'], concat['좌표Y'])

# 지하철 거리

In [5]:
subway_path = "../data/subway_feature.csv"
subway_data = pd.read_csv(subway_path)

In [ ]:
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial import KDTree

apartment_coords = concat[['좌표Y', '좌표X']].values
station_coords = subway_data[['위도', '경도']].values

def latlon_to_cartesian(lat, lon):
    R = 6371.0  # 지구 반지름 (km)
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return x, y, z

apartment_cartesian = np.array([latlon_to_cartesian(lat, lon) for lat, lon in apartment_coords])
station_cartesian = np.array([latlon_to_cartesian(lat, lon) for lat, lon in station_coords])

station_tree = KDTree(station_cartesian)
nearest_distances, nearest_station_indices = station_tree.query(apartment_cartesian, k=1)

concat['근처지하철역'] = [subway_data.iloc[i]['역사명'] for i in nearest_station_indices]
concat['지하철거리'] = nearest_distances

concat[['근처지하철역', '지하철거리']].head()


In [ ]:
concat.tail()

In [ ]:
concat['좌표Y'].isnull().sum()

# 학교

In [18]:
school_path = "../data/school_data.csv"
school_data = pd.read_csv(school_path)


In [ ]:
school_data

In [31]:
school_coords = np.array(school_data[['위도', '경도']].values)

apartment_coords = np.array(concat[['좌표Y', '좌표X']].values)

school_tree = KDTree(school_coords)

nearby_school_counts = []
radius = 0.5  # 반경 1km (단위: km)

for apartment in apartment_coords:
    indices = school_tree.query_ball_point(apartment, r=radius)
    nearby_school_counts.append(len(indices))

In [33]:
concat['근처학교수'] = nearby_school_counts

In [ ]:
concat['근처학교수'].value_counts()

In [ ]:
print(concat[['좌표Y', '좌표X', '근처학교수']].head())

In [ ]:
print(school_data[['위도', '경도']].drop_duplicates().head(10))
print(school_data[['위도', '경도']].describe())


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(apartment_coords[:, 1], apartment_coords[:, 0], s=1, label='아파트')
plt.scatter(school_coords[:, 1], school_coords[:, 0], s=1, label='학교', color='red')
plt.legend()
plt.title('아파트와 학교 좌표 분포')
plt.xlabel('경도')
plt.ylabel('위도')
plt.show()


# 학군

In [ ]:
middleschool_path = "../data/seoul_middle_school_with_coords.csv"
highschool_path = "../data/seoul_high_school_with_coords.csv"
middleschool_data = pd.read_csv(middleschool_path)
highschool_data = pd.read_csv(highschool_path)

print(middleschool_data)
print(highschool_data)

In [ ]:
middleschool_data.isnull().sum()

In [ ]:
middleschool_data['등급'] = pd.qcut(
    middleschool_data['순위'], 
    q=[0, 0.1, 0.3, 0.5, 1.0], 
    labels=['A', 'B', 'C', 'D'] 
)

highschool_data['등급'] = pd.qcut(
    highschool_data['순위'],  
    q=[0, 0.1, 0.3, 0.5, 1.0],
    labels=['A', 'B', 'C', 'D'] 
)

print(middleschool_data[['학교명', '순위', '등급']].head())
print(highschool_data[['학교명', '순위', '등급']].head())


In [ ]:
middleschool_data['등급 점수'] = middleschool_data['등급'].map({'A': 3, 'B': 2, 'C': 1, 'D': 0})
highschool_data['등급 점수'] = highschool_data['등급'].map({'A': 3, 'B': 2, 'C': 1, 'D': 0})

print(middleschool_data[['학교명', '순위', '등급', '등급 점수']].head())
print(highschool_data[['학교명', '순위', '등급', '등급 점수']].head())


In [ ]:
print(middleschool_data[['위도', '경도']].isnull().sum())  
print(middleschool_data[['위도', '경도']].describe()) 


In [54]:
middleschool_data['등급 점수'] = middleschool_data['등급 점수'].astype(int)
highschool_data['등급 점수'] = highschool_data['등급 점수'].astype(int)


In [ ]:
correlation = concat[['학군 점수', '아파트 가격']].corr()
print(correlation)


In [ ]:
combined_data = pd.concat([middleschool_data, highschool_data])
a_grade_schools = combined_data[combined_data['등급'] == 'A']
print(a_grade_schools[['학교명', '순위', '등급', '등급 점수']].head())


In [ ]:
combined_data['구'] = combined_data['위치'].apply(lambda x: x.split(' ')[0] if pd.notna(x) else None)
combined_data['동'] = combined_data['위치'].apply(lambda x: x.split(' ')[1] if pd.notna(x) else None)

print(combined_data[['위치', '구', '동']].head())


In [ ]:
no_dong_data = combined_data[~combined_data['위치'].str.contains(r'동', regex=True)]

In [ ]:
filtered_data = combined_data[combined_data['위치'].str.contains(r'\d+|가', regex=True)]

In [ ]:
import re

dong_names = [re.sub(r'\d+', '', dong) for dong in combined_data['동']]
dong_list = combined_data['동'].value_counts()
dong_list

In [ ]:
grade_counts_by_gu = combined_data.groupby(['동', '등급']).size().reset_index(name='개수')
grade_ranked = grade_counts_by_gu.sort_values(by=['등급', '개수'], ascending=[True, False])
grade_ranked

In [ ]:
top_n = grade_ranked.groupby('등급').head(5)
top_n

In [ ]:
apartment_lat = concat['좌표Y']
apartment_lon = concat['좌표X']

latitude = a_grade_schools['위도']
longitude = a_grade_schools['경도']

plt.figure(figsize=(12, 8))

plt.scatter(apartment_lon, apartment_lat, s=1, label='아파트', alpha=0.5, color='blue')

plt.scatter(longitude, latitude, c='red', label='A 등급 학교', alpha=0.7)

plt.title('아파트와 A 등급 학교 좌표 시각화')
plt.xlabel('경도')
plt.ylabel('위도')
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
district_counts = a_grade_schools['위치'].value_counts()

district_counts.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('구별 A 등급 학교 분포')
plt.xlabel('구')
plt.ylabel('학교 수')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


In [146]:
school_coords = np.array(combined_data[['위도', '경도']].values)
apartment_coords = np.array(concat[['좌표Y', '좌표X']].values)

In [147]:
apartment_cartesian = np.array([latlon_to_cartesian(lat, lon) for lat, lon in apartment_coords])
school_cartesian = np.array([latlon_to_cartesian(lat, lon) for lat, lon in school_coords])

school_tree = KDTree(school_cartesian)
nearest_distances, nearest_school_indices = school_tree.query(apartment_cartesian, k=1)

concat['근처학교'] = [combined_data.iloc[i]['학교명'] for i in nearest_school_indices]
concat['학교거리'] = nearest_distances



In [ ]:
concat[['근처학교', '학교거리']].value_counts()

In [ ]:
combined_data[['학교명', '등급']]

In [ ]:
concat

In [ ]:
concat['학교등급'] = concat['근처학교'].map(
    combined_data.set_index('학교명')['등급']
)

concat['학교등급']

In [ ]:
a_school_data = concat[concat['등급'] == 'A']
a_school_data

# 백화점

In [ ]:
commercial_district_path = "../data/commercial_district.csv"
commercial_district_path  = pd.read_csv(commercial_district_path)
commercial_district_path.head()

In [ ]:
commercial_district_path[['도로명주소', '업태구분명']]

In [ ]:
filtered_data = commercial_district_path[
    commercial_district_path['사업장명'].str.contains(
        '갤러리아|롯데|신세계|현대', case=False, na=False
    ) & commercial_district_path['사업장명'].str.contains('백화점', case=False, na=False)
]

print(filtered_data.shape)

filtered_data[['지번주소', '도로명주소', '사업장명', '좌표정보(X)',	'좌표정보(Y)']]

In [ ]:
def extract_gu_dong(row):
    if pd.notna(row['도로명주소']):
        gu = row['도로명주소'].split(' ')[1]
        dong = row['도로명주소'].split('(')[-1].split(')')[0][:3]
        return gu, dong
    elif pd.notna(row['지번주소']):
        gu = row['지번주소'].split(' ')[1]
        dong = row['지번주소'].split(' ')[2]
        return gu, dong
    else:
        return None, None

filtered_data[['구', '동']] = filtered_data.apply(lambda row: pd.Series(extract_gu_dong(row)), axis=1)

filtered_data[['도로명주소', '지번주소', '구', '동']].head()

In [ ]:
filtered_data[['구', '동']].isnull().sum()

In [ ]:
from pyproj import CRS, Transformer

tm_crs = CRS("EPSG:5181")  # TM 좌표계
wgs_crs = CRS("EPSG:4326")  # WGS84 좌표계

transformer = Transformer.from_crs(tm_crs, wgs_crs, always_xy=True)

def convert_coordinates(x, y):
    lon, lat = transformer.transform(x, y)
    return lat, lon

filtered_data['위도'], filtered_data['경도'] = zip(
    *filtered_data.apply(lambda row: convert_coordinates(row['좌표정보(X)'], row['좌표정보(Y)']), axis=1)
)

filtered_data[['사업장명', '도로명주소', '위도', '경도']].head()


In [ ]:
import matplotlib.pyplot as plt

apartment_lat = concat['좌표Y']
apartment_lon = concat['좌표X']

latitude = filtered_data['위도']
longitude = filtered_data['경도']

plt.figure(figsize=(12, 8))

plt.scatter(apartment_lon, apartment_lat, s=1, label='아파트', alpha=0.5, color='blue')

plt.scatter(longitude, latitude, c='red', label='백화점', alpha=0.7)

plt.title('아파트와 백화점 좌표 시각화')
plt.xlabel('경도')
plt.ylabel('위도')
plt.legend()
plt.grid(True)

plt.show()


# 저장

In [ ]:
concat.columns

In [ ]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat.query('is_test==0')
dt_test = concat.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

In [ ]:
dt_test.drop(columns='target', inplace=True)
print(dt_train.shape, dt_test.shape)

In [167]:
dt_train.to_csv('./dis_train.csv', index=False)
dt_test.to_csv('./dis_test.csv', index=False)